In [1]:
import pandas as pd
import pandasql as psql
import numpy as np
import json
from shapely.geometry import Point
from shapely.geometry import Polygon

In [6]:
neigh_bounds = '../SimplifiedGeoJson/nbounds_by_boro.json'
pct_bounds = '../SimplifiedGeoJson/precinct_bounds_by_borough.json'

In [7]:
with open(neigh_bounds, 'r') as fp:
    nbounds = json.load(fp)

In [5]:
with open(pct_bounds, 'r') as fp:
    pbounds = json.load(fp)

### Get neighbourhood name for listings currently listed as "New Dev"

In [ ]:
def area_form_coord(long, lat, boro):
    pt = Point(long, lat)

    for neigh in nbounds[boro]:
        ncoords = neigh['coords'][0][0]
        coords = [tuple(x) for x in ncoords]
        poly = Polygon(coords)

        if poly.contains(pt):
            return neigh['name']
    
    return None

#### Brooklyn

In [3]:
bk_df = pd.DataFrame(pd.read_csv('bklyn_df.csv'))

#### Bronx

#### Manhattan

#### Queens

#### Staten-Island